In [1]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
from copy import deepcopy
import os
import colour

In [2]:
folder_carne = "imagenes_carne"
folder_paleta = "imagenes_paletas"

files = os.listdir(folder_carne)

In [3]:
# Function for finding the circles on an image
def find_circles(paleta):
    gray = cv2.cvtColor(paleta, cv2.COLOR_RGB2GRAY)
    detected_circles = cv2.HoughCircles(gray, 
                        cv2.HOUGH_GRADIENT, 1, 20, param1 = 100,
                    param2 = 25, minRadius = 1, maxRadius = 25)

    if detected_circles is not None:
        # Convert the circle parameters a, b and r to integers.
        detected_circles = np.uint16(np.around(detected_circles))

    return detected_circles[0]

In [4]:
# Function for sorting the detected circles on the palette and getting the median of their colors
def getColorsMatrix(paleta,detected_circles):
    my_circles = deepcopy(detected_circles)
    circles_column_idx = np.argsort(my_circles[:,0])
    circ_sorted_x = deepcopy(my_circles[circles_column_idx,:])

    # First we split the circles array into 4 sections
    circ_split = np.array_split(circ_sorted_x,4)

    # Now we sort each section based on y coordinates
    circ_sorted_y = np.zeros_like(circ_split)
    for k in range(len(circ_split)):
        circles_row_idx = np.argsort(circ_split[k][:,1])
        circ_sorted_y[k] = deepcopy(circ_split[k][circles_row_idx,:])
    
    # And finally we reshape the array so it matches the median array
    rows,cols,_ = circ_sorted_y.shape
    my_circles = np.reshape(circ_sorted_y,(rows*cols,_))

    median_array = np.zeros((len(my_circles),3))
    for k in range(len(my_circles)):
        (x,y,r) = my_circles[k]
        
        median_r = np.median(paleta[y-(r//2):y+(r//2),x-(r//2):x+(r//2),0])
        median_g = np.median(paleta[y-(r//2):y+(r//2),x-(r//2):x+(r//2),1])
        median_b = np.median(paleta[y-(r//2):y+(r//2),x-(r//2):x+(r//2),2])
        
        median_array[k] = (median_r,median_g,median_b)
    
    return my_circles,median_array

In [5]:
methods = sorted(colour.COLOUR_CORRECTION_METHODS.keys())
methods

['Cheung 2004', 'Finlayson 2015', 'Vandermonde']

In [6]:
for method in methods:
    for file in files:
        img = cv2.imread(folder_carne+f"\\{file}")
        img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)

        img_paleta = cv2.imread(folder_paleta+f"\\{file}")
        img_paleta = cv2.cvtColor(img_paleta,cv2.COLOR_BGR2RGB)

        # print(f"Imagenes leidas: {file}")
        # plt.figure(figsize=(15,5))
        # plt.subplot(1,2,1)
        # plt.imshow(img)
        # plt.subplot(1,2,2)
        # plt.imshow(img_paleta)
        # plt.show()

        detected_circles = find_circles(img_paleta)
        # print(f"Numero de circulos de la paleta de muestra encontrados: {detected_circles.shape}")
        # copy = img_paleta.copy()
        # for (x,y,r) in detected_circles:
            
        #     # Draw the circumference of the circle.
        #     cv2.circle(copy, (x, y), r, (0, 255, 255), 2)

        #     # Draw a small circle (of radius 1) to show the center.
        #     cv2.circle(copy, (x, y), 1, (100, 255, 100), 2)

        # print("Circulos de la paleta de muestra detectados")
        # plt.figure(figsize=(5,5))
        # plt.imshow(copy)
        # plt.show()

        # canvas_p = np.zeros_like(img_paleta)
        circles_p,median_p = getColorsMatrix(img_paleta,detected_circles)
        # for k in range(len(circles_p)):
        #     (x,y,r) = circles_p[k]
        #     color = median_p[k]
        #     cv2.circle(canvas_p,(x,y),r,color,-1)

        # print("medianas de los colores de la paleta de muestra encontradas")
        # plt.figure(figsize=(10,10))
        # plt.subplot(1,2,1)
        # plt.imshow(img_paleta)
        # plt.subplot(1,2,2)
        # plt.imshow(canvas_p)
        # plt.show()

        paleta_calibracion = cv2.imread(folder_paleta+"/CALIBRACION.JPG")
        paleta_calibracion = cv2.cvtColor(paleta_calibracion,cv2.COLOR_BGR2RGB)

        # print("Leer paleta de calibracion")
        # plt.figure(figsize=(5,5))
        # plt.imshow(paleta_calibracion)
        # plt.show()

        detected_circles = find_circles(paleta_calibracion)
        # print(f"Numero de circulos de la paleta de calibracion encontrados: {detected_circles.shape}")
        # copy = paleta_calibracion.copy()  
        # for (x,y,r) in detected_circles:
            
        #     # Draw the circumference of the circle.
        #     cv2.circle(copy, (x, y), r, (0, 255, 255), 2)

        #     # Draw a small circle (of radius 1) to show the center.
        #     cv2.circle(copy, (x, y), 1, (100, 255, 100), 2)

        # print("Circulos de la paleta de calibracion detectados")
        # plt.figure(figsize=(5,5))
        # plt.imshow(copy)
        # plt.show()

        # canvas_cal = np.zeros_like(paleta_calibracion)
        circles_cal,median_cal = getColorsMatrix(paleta_calibracion,detected_circles)
        # for k in range(len(circles_cal)):
        #     (x,y,r) = circles_cal[k]
        #     color = median_cal[k]
        #     cv2.circle(canvas_cal,(x,y),r,color,-1)

        # print("medianas de los colores de la paleta de calibracion encontradas")
        # plt.figure(figsize=(10,10))
        # plt.subplot(1,2,1)
        # plt.imshow(paleta_calibracion)
        # plt.subplot(1,2,2)
        # plt.imshow(canvas_cal)
        # plt.show()

        # las medianas deben estar como flotantes
        median_p,median_cal = median_p.astype("float"), median_cal.astype("float")

        img_copy = img.copy()
        height,width,chan = img_copy.shape
        vec_img = np.reshape(img_copy,(height*width,chan)).astype("float")

        corrected_vec = colour.colour_correction(vec_img,median_p,median_cal,method=method)

        # Hacer el clip (que no salga de los valores limite)
        corrected_vec[corrected_vec > 255],corrected_vec[corrected_vec < 0] = 255,0
        
        corrected_img = np.uint8(np.reshape(corrected_vec,(height,width,chan)))

        paleta_copy = img_paleta.copy()
        h,w,ch = paleta_copy.shape
        vec_paleta = np.reshape(paleta_copy,(h*w,ch)).astype("float")

        corrected_paleta = colour.colour_correction(vec_paleta,median_p,median_cal,method=method)

        # Hacer el clip (que no salga de los valores limite)
        corrected_paleta[corrected_paleta > 255],corrected_paleta[corrected_paleta < 0] = 255,0
        
        corrected_paleta = np.uint8(np.reshape(corrected_paleta,(h,w,ch)))
        
        cv2.imwrite(f"colour_correction/imagenes_carne/{str(method)}/{file}",cv2.cvtColor(corrected_img,cv2.COLOR_RGB2BGR))
        cv2.imwrite(f"colour_correction/imagenes_paleta/{str(method)}/{file}",cv2.cvtColor(corrected_paleta,cv2.COLOR_RGB2BGR))

        # plt.figure(figsize=(20,10))
        # plt.subplot(1,2,1)
        # plt.title("Original image")
        # plt.imshow(img)
        # plt.subplot(1,2,2)
        # plt.title("Calibrated image")
        # plt.imshow(corrected_img)
        # plt.show()
